In [1]:
# Roman Ramirez, rr8rk@virignia.edu
# Neo4j

In [23]:
# IMPORTING MODULES

import pprint
import pandas as pd
from py2neo import Graph
from py2neo import Node
from py2neo import Relationship
from py2neo import NodeMatcher

## SETUP

In [3]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password"))

In [4]:
STUDENT_DATABASE = 'student_database'
SUBJECT_DATABASE = 'subject_database'
FACULTY_DATABASE = 'faculty_database'
HOUSE_DATABASE = 'house_database'

# defining node types
STUDENT = 'Student'
HOUSE = 'House'
YEAR = 'Year'
COLOR = 'Color'
CLASS = 'Class'
FACULTY = 'Faculty'
SUBJECT = 'Subject'
SCHOOL = 'School'

# reading all sheets
student_df = pd.read_excel('database_draft.xlsx', sheet_name=STUDENT_DATABASE)
subject_df = pd.read_excel('database_draft.xlsx', sheet_name=SUBJECT_DATABASE)
faculty_df = pd.read_excel('database_draft.xlsx', sheet_name=FACULTY_DATABASE)
house_df = pd.read_excel('database_draft.xlsx', sheet_name=HOUSE_DATABASE)

# getting all unique values
student_uniques = dict()
subject_uniques = dict()
faculty_uniques = dict()
house_uniques = dict()

for col in student_df.columns:
    student_uniques[col] = set(student_df[col])
for col in subject_df.columns:
    subject_uniques[col] = set(subject_df[col])
for col in faculty_df.columns:
    faculty_uniques[col] = set(faculty_df[col])
for col in house_df.columns:
    house_uniques[col] = set(house_df[col])

## Creating and adding to Neo4j

In [5]:
student_df.head()

,_id,student_fname,student_lname,school,house_id,role,year,fav_color,fav_class_id
0,hp215,harry,potter,hogwarts,0.0,student,4.0,red,0.0
1,rw912,ron,weasley,hogwarts,0.0,student,4.0,blue,0.0
2,hg017,hermione,granger,hogwarts,0.0,student,4.0,orange,1.0
3,dm120,draco,malfoy,hogwarts,3.0,student,4.0,purple,5.0
4,nl742,neville,longbottom,hogwarts,0.0,student,4.0,red,4.0


In [6]:
faculty_df.head()

,faculty_id,faculty_fname,faculty_lname,subject_id,school,house_id
0,111.0,minerva,mcgonagall,4.0,hogwarts,0.0
1,222.0,severus,snape,5.0,hogwarts,3.0
2,333.0,filius,flitwick,13.0,hogwarts,1.0
3,444.0,pomona,sprout,2.0,hogwarts,2.0
4,555.0,sophie,lebovitz,10.0,beauxbatons,6.0


In [7]:
house_df.head()

,house_id,name,color_1,color_2,value_1,value_2,value_3,value_4,house_points
0,0,gryffindor,red,gold,courage,bravery,nerve,chivalry,1170.0
1,1,ravenclaw,blue,bronze,hard work,patience,justice,loyalty,1350.0
2,2,hufflepuff,yellow,black,intelligence,learning,wisdom,wit,1240.0
3,3,slytherin,green,silver,ambition,cunning,leadership,resourcefulness,1436.0
4,4,ombrelune,gray,NaN,cunning,logic,ambition,curiosity,1334.0


In [8]:
subject_df.head()

,class_id,class_name,capacity,location,faculty_id
0,0.0,defense,20.0,3C,222.0
1,1.0,arithmancy,15.0,turris_magnus,111.0
2,2.0,herbology,10.0,greenhouse,333.0
3,3.0,flying,20.0,field,444.0
4,4.0,transfigurations,60.0,1B,111.0


In [9]:
# creating nodes

graph.run("MATCH (n) DETACH DELETE n")

student_df.head()

### LARGE NODES

# making all student nodes
nodes_student = dict()
for i in range(len(student_df)):
    row = student_df.iloc[i]
    local_node = Node(
        STUDENT,
        _id=row['_id'],
        fname=row['student_fname'],
        lname=row['student_lname'],
        name=f"{row['student_fname'].capitalize()} {row['student_lname'].capitalize()}"
    )
    nodes_student.update({row['_id']: local_node})
    graph.create(local_node)
    
# making all faculty nodes
nodes_faculty = dict()
for i in range(len(faculty_df)):
    row = faculty_df.iloc[i]
    local_node = Node(
        FACULTY,
        _id=int(row['faculty_id']),
        fname=row['faculty_fname'],
        lname=row['faculty_lname'],
        name=f"{row['faculty_fname'].capitalize()} {row['faculty_lname'].capitalize()}",
    )
    nodes_faculty.update({row['faculty_id']: local_node})
    graph.create(local_node)
    
# making all subject nodes
nodes_subject = dict()
for i in range(len(subject_df)):
    row = subject_df.iloc[i]
    local_node = Node(
        SUBJECT,
        _id=int(row['class_id']),
        name=row['class_name'],
        capacity=int(row['capacity']),
        location=row['location']
    )
    nodes_subject.update({row['class_id']: local_node})
    graph.create(local_node)
    
# making all house nodes
nodes_house = dict()
for i in range(len(house_df)):
    row = house_df.iloc[i]
    local_node = Node(
        HOUSE,
        house_id=int(row['house_id']),
        name=row['name'],
        color_1=row['color_1'],
        color_2=row['color_2'],
        value_1=row['value_1'],
        value_2=row['value_2'],
        value_3=row['value_3'],
        value_4=row['value_4'],
        house_points=int(row['house_points'])
    )
    nodes_house.update({row['house_id']: local_node})
    graph.create(local_node)
    
### SMALL ONES

# making all schools (students)
nodes_school = dict()
for school in student_uniques['school']:
    local_node = Node(
        SCHOOL,
        school=school
    )
    graph.create(local_node)
    nodes_school.update({school: local_node})
    
# making all years (students)
nodes_year = dict()
for year in student_uniques['year']:
    local_node = Node(
        YEAR,
        year=f"Year {int(year)}"
    )
    graph.create(local_node)
    nodes_year.update({year: local_node})

# making all color nodes (students)
nodes_color = dict()
for color in student_uniques['fav_color']:
    local_node = Node(
        COLOR,
        fav_color=color
    )
    graph.create(local_node)
    nodes_color.update({color: local_node})
    
# creating relationships

# defining relationships

student2subject = Relationship.type("LIKES_THE_SUBJECT")
student2house   = Relationship.type("BELONGS_TO")
student2school  = Relationship.type("ATTENDS")
student2year    = Relationship.type("IS_YEAR")
student2color   = Relationship.type("LIKES_THE_COLOR")

faculty2subject = Relationship.type("TEACHES")
faculty2house   = Relationship.type("BELONGS_TO")
faculty2school  = Relationship.type("TEACHES_AT")

def assign_all_relationships(d1, n1, d2, n2, r):
    for (a, b) in zip(d1, d2):
        graph.create(r(n1[a], n2[b]))

assign_all_relationships(student_df['_id'], nodes_student, student_df['fav_class_id'], nodes_subject, student2subject)
assign_all_relationships(student_df['_id'], nodes_student, student_df['house_id'], nodes_house, student2house)
assign_all_relationships(student_df['_id'], nodes_student, student_df['school'], nodes_school, student2school)
assign_all_relationships(student_df['_id'], nodes_student, student_df['year'], nodes_year, student2year)
assign_all_relationships(student_df['_id'], nodes_student, student_df['fav_color'], nodes_color, student2color)

assign_all_relationships(faculty_df['faculty_id'], nodes_faculty, faculty_df['subject_id'], nodes_subject, faculty2subject)
assign_all_relationships(faculty_df['faculty_id'], nodes_faculty, faculty_df['house_id'], nodes_house, faculty2house)
assign_all_relationships(faculty_df['faculty_id'], nodes_faculty, faculty_df['school'], nodes_school, faculty2school)

# QUERIES

In [10]:
matcher = NodeMatcher(graph)

In [11]:
matcher.match("Student", name="Hannah Abbott").all()
graph.run("MATCH (n: Student) RETURN n")

n
"(_37:Student {_id: 'hp215', fname: 'harry', lname: 'potter', name: 'Harry Potter'})"
"(_38:Student {_id: 'rw912', fname: 'ron', lname: 'weasley', name: 'Ron Weasley'})"
"(_39:Student {_id: 'hg017', fname: 'hermione', lname: 'granger', name: 'Hermione Granger'})"


In [71]:
# number of students in each house
for house in house_uniques['name']:
    local_query = "MATCH (n: House {name: \"%s\"})<--(s: Student) RETURN count(s) as %s_students" % (house, house)
    local_query_result = graph.run(local_query)
    print(local_query_result)

 vulpelara_students 
--------------------
                  0 

 bellefeuille_students 
-----------------------
                     1 

 ombrelune_students 
--------------------
                  1 

 soscrofa_students 
-------------------
                 1 

 hufflepuff_students 
---------------------
                   5 

 ravenclaw_students 
--------------------
                  7 

 slytherin_students 
--------------------
                  8 

 ucilena_students 
------------------
                0 

 gryffindor_students 
---------------------
                  21 

 papillonlisse_students 
------------------------
                      1 



In [126]:
# number of students in each house
local_query = '''
MATCH p=(a:Student)-[]-(b:House)
RETURN DISTINCT a.name, b.name
'''
local_query_result = graph.run(local_query)
local_query_result.data()

[{'a.name': 'Oliver Wood', 'b.name': 'gryffindor'},
 {'a.name': 'Ginny Weasley', 'b.name': 'gryffindor'},
 {'a.name': 'George Weasley', 'b.name': 'gryffindor'},
 {'a.name': 'Fred Weasley', 'b.name': 'gryffindor'},
 {'a.name': 'Romilda Vane', 'b.name': 'gryffindor'},
 {'a.name': 'Dean Thomas', 'b.name': 'gryffindor'},
 {'a.name': 'Alicia Spinnet', 'b.name': 'gryffindor'},
 {'a.name': 'Parvati Patil', 'b.name': 'gryffindor'},
 {'a.name': 'Cormac Mclaggen', 'b.name': 'gryffindor'},
 {'a.name': 'Eloise Midgen', 'b.name': 'gryffindor'},
 {'a.name': 'Lee Jordan', 'b.name': 'gryffindor'},
 {'a.name': 'Angelina Johnson', 'b.name': 'gryffindor'},
 {'a.name': 'Seamus Finnigan', 'b.name': 'gryffindor'},
 {'a.name': 'Dennis Creevey', 'b.name': 'gryffindor'},
 {'a.name': 'Colin Creevey', 'b.name': 'gryffindor'},
 {'a.name': 'Lavender Brown', 'b.name': 'gryffindor'},
 {'a.name': 'Katie Bell', 'b.name': 'gryffindor'},
 {'a.name': 'Neville Longbottom', 'b.name': 'gryffindor'},
 {'a.name': 'Hermione Gr

In [84]:
# who likes the same subject as harry potter?
local_query = 'MATCH (hp:Student {_id: "hp215"})-[:LIKES_THE_SUBJECT]->(m)<-[:LIKES_THE_SUBJECT]-(classmate:Student) RETURN DISTINCT classmate.name as likes_the_same_subject_as_hp'
# local_query = 'MATCH p=(hp:Student {_id: "hp215"})-[:LIKES_THE_SUBJECT]->(m)<-[:LIKES_THE_SUBJECT]-(classmate:Student) RETURN p'
local_query_result = graph.run(local_query)
[x['likes_the_same_subject_as_hp'] for x in local_query_result.data()]

['Oliver Wood',
 'Ginny Weasley',
 'Fred Weasley',
 'Theodore Nott',
 'Gregory Goyle',
 'Vincent Crabbe',
 'Hannah Abbott',
 'Ron Weasley']

In [73]:
# who were harry potter's classmates?
local_query = 'MATCH (hp:Student {_id: "hp215"})-[:LIKES_THE_SUBJECT]->(m)<-[:LIKES_THE_SUBJECT]-(classmate:Student)-[:IS_YEAR]->(d:Year {year: "Year 4"}) RETURN DISTINCT classmate.name as classmates'
local_query_result = graph.run(local_query)
local_query_result.to_data_frame()

,classmates
0,Theodore Nott
1,Gregory Goyle
2,Vincent Crabbe
3,Hannah Abbott
4,Ron Weasley


In [135]:
# closest relationship between hermione granger and sophie lebovitz
local_query = '''
MATCH p=shortestPath(
(a:Student {name:"Hermione Granger"})-[*]-(b:Faculty {name:"Sophie Lebovitz"})
)
RETURN p
'''
local_query_result = graph.run(local_query)
for point in local_query_result.to_subgraph():
    print(point)

(Hermione Granger)-[:IS_YEAR {}]->(_123)
(Parvati Patil)-[:IS_YEAR {}]->(_123)
(Parvati Patil)-[:LIKES_THE_COLOR {}]->(_128)
(Gabrielle Delacour)-[:LIKES_THE_COLOR {}]->(_128)
(Gabrielle Delacour)-[:ATTENDS {}]->(_117)
(Sophie Lebovitz)-[:TEACHES_AT {}]->(_117)


In [ ]:
# number of classes in each school
local_query = '''
MATCH p=(a:Subject)-[]-()-[]-(c:School)
RETURN DISTINCT a._id, c.school, a.name 
'''
local_query_result = graph.run(local_query)
local_query_result.data()

[{'a._id': 12, 'c.school': 'beauxbatons', 'a.name': 'etiquette'},
 {'a._id': 11, 'c.school': 'beauxbatons', 'a.name': 'literature'},
 {'a._id': 10, 'c.school': 'beauxbatons', 'a.name': 'culinary_arts'},
 {'a._id': 9, 'c.school': 'durmstrang', 'a.name': 'rare_languages'},
 {'a._id': 0, 'c.school': 'durmstrang', 'a.name': 'defense'},
 {'a._id': 8, 'c.school': 'durmstrang', 'a.name': 'spellworks'},
 {'a._id': 2, 'c.school': 'hogwarts', 'a.name': 'herbology '},
 {'a._id': 13, 'c.school': 'hogwarts', 'a.name': 'charms'},
 {'a._id': 5, 'c.school': 'hogwarts', 'a.name': 'potions'},
 {'a._id': 4, 'c.school': 'hogwarts', 'a.name': 'transfigurations'},
 {'a._id': 0, 'c.school': 'hogwarts', 'a.name': 'defense'},
 {'a._id': 3, 'c.school': 'hogwarts', 'a.name': 'flying'},
 {'a._id': 7, 'c.school': 'hogwarts', 'a.name': 'quidditch'},
 {'a._id': 6, 'c.school': 'hogwarts', 'a.name': 'lunch'},
 {'a._id': 1, 'c.school': 'hogwarts', 'a.name': 'arithmancy'}]

In [134]:
# showing the students' favorite classes and also showing who its taught by
local_query = '''
MATCH p=(a:Faculty)-[b:TEACHES]-(c:Subject)-[d:LIKES_THE_SUBJECT]-(e:Student)
RETURN DISTINCT e.name, c.name, a.name
'''
local_query_result = graph.run(local_query)
pprint.pprint(local_query_result.data())


[{'a.name': 'Minerva Mcgonagall',
  'c.name': 'transfigurations',
  'e.name': 'Blaise Zabini'},
 {'a.name': 'Minerva Mcgonagall',
  'c.name': 'transfigurations',
  'e.name': 'Romilda Vane'},
 {'a.name': 'Minerva Mcgonagall',
  'c.name': 'transfigurations',
  'e.name': 'Dean Thomas'},
 {'a.name': 'Minerva Mcgonagall',
  'c.name': 'transfigurations',
  'e.name': 'Parvati Patil'},
 {'a.name': 'Minerva Mcgonagall',
  'c.name': 'transfigurations',
  'e.name': 'Lee Jordan'},
 {'a.name': 'Minerva Mcgonagall',
  'c.name': 'transfigurations',
  'e.name': 'Seamus Finnigan'},
 {'a.name': 'Minerva Mcgonagall',
  'c.name': 'transfigurations',
  'e.name': 'Cedric Diggory'},
 {'a.name': 'Minerva Mcgonagall',
  'c.name': 'transfigurations',
  'e.name': 'Susan Bones'},
 {'a.name': 'Minerva Mcgonagall',
  'c.name': 'transfigurations',
  'e.name': 'Katie Bell'},
 {'a.name': 'Minerva Mcgonagall',
  'c.name': 'transfigurations',
  'e.name': 'Neville Longbottom'},
 {'a.name': 'Severus Snape', 'c.name': 'pot